In [1]:
#https://en.wikipedia.org/wiki
import requests
from bs4 import BeautifulSoup
import re
import json
import pickle
import datetime
    

In [2]:
def name_formatter(name):
    name = re.sub("_", " ", name).strip().title()
    return(name)

In [6]:
name_formatter("Mohammad_Najibullah")

'Mohammad Najibullah'

In [3]:
def ar_lookup(name):
    """
    Convert names from English to Arabic using European Media Monitors and 
     code modified from Phil Schrodt.
    """
    name = name_formatter(name)
    base_url = "http://emm.newsexplorer.eu/NewsExplorer/search/en/entities?query="
    url = base_url + name
    
    try:
        page = requests.get(url)
        soup = BeautifulSoup(page.content, "lxml")
        name_url = soup.find("p", {"class" : "center_headline"}).find("a")['href']
    except Exception as e:
        #print("Couldn't get page of results back: ", e)
        return []

    try:
        base = "http://emm.newsexplorer.eu/NewsExplorer/search/en/"
        name_url = base + name_url
        name_page = requests.get(name_url)
        soup = BeautifulSoup(name_page.content, "lxml")
        # check to make sure in list???? Take in alt names???/petrarch2/petrarch2/data/dictionaries
        names = soup.find("td", {"colspan" : "1"}).find_all("p")
        names = [i.text for i in names][1:]
        names_en = [i for i in names if re.search("\(.*?Eu|\(.*?en", i)]
        names_en = [re.sub("\s+?\(.+?\)", "", name) for name in names_en]
        #print("Found match. Matched English name: ", names_en[0])
        names_ar = [i for i in names if re.search("\(.*?ar", i)]
        names_ar = [re.sub("\s+?\(.+?\)", "", name) for name in names_ar]
        return names_ar
    except Exception:
        traceback.print_exc()
        return []

In [37]:
#this will need the user to pass in an english name and come back with an arabic name
def hack_wiki(eng_name):
    base_url="https://en.wikipedia.org/wiki/"+eng_name
    try:
        page=requests.get(base_url)
        soup=BeautifulSoup(page.content,"lxml")
        name=soup.find(id="firstHeading").contents
        ar_url=soup.find("li",{"class":"interwiki-ar"}).find("a")['href']
        #print(ar_url)
        ar_page=requests.get(ar_url)
        ar_soup=BeautifulSoup(ar_page.content,"lxml")
        ar_name=ar_soup.find(id="firstHeading").contents
        print(ar_name)
        #print("name "+name+" url: "+str(ar_url))
    except Exception as e:
        print(e)

In [38]:
hack_wiki("Mohammad_Najibullah")

['محمد نجيب الله']
